# Agent

Large Language Models (LLMs) possess significant power but encounter inherent limitations that hinder their practical applications. Agents serve as intermediaries, addressing these limitations and connecting LLMs with real-world scenarios. LLMs' complexity is streamlined by agents, breaking down tasks into manageable steps, enhancing user experience, and enabling more efficient task execution. Agents facilitate LLMs' interaction with the real world by providing access to external data and resources, extending their capabilities beyond language processing. Integration with external tools expands LLM capabilities, allowing them to handle a broader range of tasks. Agents also mitigate errors by offering continuous feedback, aiding in performance improvement. Additionally, agents automate LLM management tasks, such as deployment and monitoring, reducing user burden and optimizing LLM usage. In summary, agents play a crucial role in overcoming LLM limitations, enabling more effective and comprehensive operation to address real-world challenges and transform industries.

Agents in LangChain contain a toolbox that allows them to solve a variety of problems. They can call upon different tools based on the task at hand to provide users with comprehensive and informative solutions. Agents can generate creative text, access and process information from external sources, translate languages, summarize text, and identify patterns in data. As more tools are added to the toolbox, agents will become even more powerful and versatile.

* [langchain agents turorail](https://python.langchain.com/docs/modules/agents/tools/)


Langchain is a versatile platform that provides a wide array of tools specifically designed for language processing tasks. These tools are not only efficient but also user-friendly, making them ideal for both beginners and experts in the field.

What sets Langchain apart is its flexibility. If the available tools do not meet your specific requirements, Langchain gives you the unique opportunity to create your own custom tool. This means you can tailor a tool to perform exactly the task you need, making your work more effective and streamlined.

Whether you're working on a complex linguistic project or simply need to automate a routine task, Langchain has got you covered. Its combination of ready-to-use tools and the ability to create custom ones makes it a one-stop solution for all your language processing needs.

* [available tools](https://python.langchain.com/docs/integrations/tools)


## Initialization section

In [12]:
!pip install langchain
!pip install openai==v0.28.1
!pip install -U duckduckgo-search
!pip install langchainhub
! pip install youtube_search



## Import section

In [13]:
import os


from langchain.chat_models import ChatOpenAI

from langchain.prompts import ChatPromptTemplate

from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import YouTubeSearchTool



from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

from langchain.agents import tool,Tool



from langchain.agents import AgentType, initialize_agent



## preparation

In [3]:
openai_api_key = ...
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
llm = ChatOpenAI(temperature=0)

## First task
We are going to create a program that will answer question base on duckduckgo search engine.

In [5]:
search = DuckDuckGoSearchRun()

In [6]:
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="You can use this tool when you need information about know",
    ),
]

we are going to download one of promt template from [hub](https://smith.langchain.com/hub)

In [7]:

prompt = hub.pull("hwchase17/react-chat")

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [8]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)



In [11]:
agent.invoke({"input": "price of gold today in USA "})["output"]



> Entering new AgentExecutor chain...
I need to find the current price of gold in the USA.
Action: Search
Action Input: "current price of gold in USA"
Observation: We use the last quote at 5:00 PM as the close of that given day. Change is always the difference between the current price and the price at 5:00pm. For example: Gold last traded at $1,200 at 5:00 PM on January 17. If it is January 17 at 6:30 PM and the price is $1,202, we will show a change of +2.00. Live 24-hour Gold Price Spot Chart from New York, London, Hong Kong and Sydney. Gold Prices Updated Every Minute. ... Current Lease Rates. All Metals Gold (small) Gold (detailed) Silver (small) Silver (detailed) Platinum Palladium. Historical Gold Price Performance in USD. ... Drop us a line info@kitco.com. Gold Price in USA Today: per oz 2,032.71 United States dollars. Gold prices for the USA are denominated in American dollars (USD). Live Price of Gold provides daily updates on gold prices across various states in the United

'The price of gold today in the USA is $2,037 per ounce.'

## task 2
Create a program that can retrieve a YouTube video link and provide answers based on a search query.

In [14]:
yt_search = YouTubeSearchTool()



In [18]:
tools = [
    Tool.from_function(
        func=search.run,
        name="Search web",
        description="You can use this tool when you need information about know",
    ),
    Tool.from_function(
        func= yt_search.run,
        name="Search youtube",
        description="You can use this tool when you want to get a url of video from youtube",
    ),
]


In [19]:

prompt = hub.pull("hwchase17/react-chat")

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [20]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)



In [29]:
agent.invoke({"input": "give me gta VI trailer link"})["output"]



> Entering new AgentExecutor chain...
I need to find the trailer for GTA VI on YouTube.
Action: Search youtube
Action Input: "GTA VI trailer"
Observation: ['https://www.youtube.com/watch?v=QdBZY2fkU-0&pp=ygUOR1RBIFZJIHRyYWlsZXI%3D', 'https://www.youtube.com/watch?v=bMq4OtZTigk&pp=ygUOR1RBIFZJIHRyYWlsZXI%3D']
Thought:There are two possible links for the GTA VI trailer on YouTube.
Action: Search youtube
Action Input: "GTA VI trailer official"
Observation: ['https://www.youtube.com/watch?v=QdBZY2fkU-0&pp=ygUXR1RBIFZJIHRyYWlsZXIgb2ZmaWNpYWw%3D', 'https://www.youtube.com/watch?v=NPLKtIkOirM&pp=ygUXR1RBIFZJIHRyYWlsZXIgb2ZmaWNpYWw%3D']
Thought:I have found the official trailer for GTA VI on YouTube.
Final Answer: Here is the official trailer for GTA VI: https://www.youtube.com/watch?v=QdBZY2fkU-0

> Finished chain.


'Here is the official trailer for GTA VI: https://www.youtube.com/watch?v=QdBZY2fkU-0'

In [30]:
agent.invoke({"input": "Which currency has the highest value at the moment?"})["output"]



> Entering new AgentExecutor chain...
I should search the web to find the current highest valued currency.
Action: Search web
Action Input: "highest valued currency at the moment"
Observation: Here are the top 10 strongest currencies: 1. Kuwaiti Dinar (KWD) The Kuwaiti dinar is the strongest currency in the world, with 1 dinar buying 3.26 dollars (or, put another way, $1 equals 0.31 ... The highest-valued currency in the world is the Kuwaiti Dinar (KWD). Since it was first introduced in 1960, the Kuwaiti dinar has consistently ranked as the world's most valuable currency. The Euro is the second-most traded currency in the world and the official currency for 19 European countries. While it was worth less than the US Dollar upon its introduction in 2000, it soon became more valuable and reached as high as $1.60 during the United States' economic crisis in 2008. 1 KWD = 2.98 EUR (Kuwaiti Dinar to Euro) Today, Kuwaiti Dinar is the highest-valued currency in the world, estimating at $3.25

'The currency with the highest value at the moment is the Kuwaiti Dinar (KWD).'

## on you
Create a program that can answer questions based on information from [wikipedia](https://python.langchain.com/docs/integrations/tools/wikipedia)
